In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as ss
import openpyxl

# S-factor calculation

Energy data from DECDATA, not from Firestone.
Separated into IC-electrons, beta-minus-electrons and auger-electrons.
Using Hough-UF-model for the male skeleton first, redo and re-check calculations.

We do have the specific energy absorbtion fractions for the lumbar vertebraes in a tabular format.

27/7/2020

Better still, make it general so we can have multiple skeletal sites and nuclides, to better get a grasp of the (full) model.

Need to convert some values from the PDF, direct import in excel returns only text-strings. This can be done better

Finnished now, have to re-calculate everything but finally we have something usefull. Set up the new analysis and the leave-1-out and all of the other stuff...

# Design of S-factors.

Need a class or object that can hold the data needed for easy get/set. The class should contain:

* Name of isotope
* A set of skeletal sites (should we here use a key-ed dictionary?)
* Nested in this should be the CFs and the trabular corrections

In [2]:
class SkeletalSFactors:
    """ Class containing S-factors for skeletal sites using the phantoms 
        described in 
        
        ref1 and ref2
        
        Used to store data for different skeletal sites and further potentially
        
        """
    
    def __init__(self, 
                 isotopename, 
                 phantomsex, 
                 skeletalfactors, 
                 icrp_cf_vals,
                 trabecular_corrections):
        
        self.isotopename = isotopename
        self.phantomsex = phantomsex
        self.skeletalfactors = skeletalfactors
        self.icrp_cf_vals = icrp_cf_vals
        self.trabecular_corrections = trabecular_corrections
        
    def print_all(self):
        print("This is a class containing")
        print(self.isotopename)
        print("Patient sex: " + self.phantomsex)
        print("An the following factors:")
        print(self.skeletalfactors)
        print("End of print")


In [ ]:
VERBOSE = True

saf_input = "utils/saf_values_male_redone.xlsx"
#saf_input = "utils/saf_values_female_redone.xlsx"

main_wb = openpyxl.load_workbook(filename=saf_input)
sheet_names = main_wb.sheetnames

# Remove unwanted sheets
non_organ_sheets = ["Tissue-AM-Masses", "Ark1"]
organ_list = sheet_names

for s in non_organ_sheets:
    if s in organ_list: organ_list.remove(s)

mass_data = pd.read_excel(saf_input, sheet_name="Tissue-AM-Masses")

list_of_sites_in_mass_file = (mass_data["SiteName"].values)

if VERBOSE:

    for organ in organ_list:
        if organ in list_of_sites_in_mass_file:
            print("Organ " + organ + " is in list")
        else:
            print("Warning, names might mis-match " + organ)
            
            
# Set up pandas-frame to get the data. Lets use first one long pandas frame that can
# later be split into different objects for simplicity
# Columns are site, CF and S-factor. CF-factor should be on the form CFXX (CFXXX when 100)
# Lenght should be number of CF-factors*number of sites

SFactorFrame = pd.DataFrame()

SFactorFrameLen = len(organ_list)*10

#SFactorFrame["Sfactors"] = np.zeros(SFactorFrameLen)
#SFactorFrame["CFs"] = np.zeros(SFactorFrameLen)
#SFactorFrame["SiteName"] = np.zeros(SFactorFrameLen)

SFactorFrame["Sfactors"] = np.zeros(0)
SFactorFrame["CFs"] = np.zeros(0)
SFactorFrame["SiteName"] = np.zeros(0)

res_index = 0

CF_vector = np.array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, -1])

for organ in organ_list:
        
    if VERBOSE:
        print(organ)
    
    # Get the mass of the organ with 100 % cellularity
    mass_value = mass_data[mass_data["SiteName"] == organ]["AM-in-MST-100-CF"].values[0]
    
    # Get the CF-value for the specific site:
    
    ICRP_CF = mass_data[mass_data["SiteName"] == organ]["ICRP-CF"].values[0]/1.0e2
        
    # Now grab the desired SAF-sheet:
    
    organ_saf_values = pd.read_excel(saf_input, sheet_name=organ)
        
    energy_look_up_vector = organ_saf_values["Energy"].values
    
    # Now grab the energy-spectrum
    
    energies = pd.read_excel("utils/decdataenergies.xlsx", sheet_name="AllElectrons")

    len_of_table = len(energies)
    
    energy_absorbtion_frame = pd.DataFrame(columns=["Energy", "Y", "phi"], 
                                       index=np.arange(len_of_table))

    energy_absorbtion_frame["Energy"] = energies["E (MeV)"]
    energy_absorbtion_frame["Y"] = energies["Y (/nt)"]
    
    for cf in CF_vector:
        
        if cf == -1:
            if VERBOSE:
                print("Now doing the ICRP")
                
            cf = ICRP_CF
            CF_string = "ICRP"
            
            for i in range(len_of_table):

                energy_to_look_up = (energy_absorbtion_frame.at[i, "Energy"])

                index = (np.abs(energy_look_up_vector-energy_to_look_up)).argmin()

                phi = (organ_saf_values.at[index, CF_string])*mass_value*cf

                energy_absorbtion_frame.at[i, "phi"] = phi

            energy_per_desintegration = (np.sum(energy_absorbtion_frame["Energy"]*energy_absorbtion_frame["Y"]*energy_absorbtion_frame["phi"]))
            
            print(organ, cf, energy_per_desintegration, res_index)
            SFactorFrame.loc[res_index,("Sfactors")] = energy_per_desintegration
            SFactorFrame.loc[res_index,("CFs")] = CF_string
            SFactorFrame.loc[res_index,("SiteName")] = organ

            res_index = res_index + 1
            
        else:
    
            CF_string = ("CF" + str(int(cf*100)))

            for i in range(len_of_table):

                energy_to_look_up = (energy_absorbtion_frame.at[i, "Energy"])

                index = (np.abs(energy_look_up_vector-energy_to_look_up)).argmin()

                phi = (organ_saf_values.at[index, CF_string])*mass_value*cf

                energy_absorbtion_frame.at[i, "phi"] = phi

            energy_per_desintegration = (np.sum(energy_absorbtion_frame["Energy"]*energy_absorbtion_frame["Y"]*energy_absorbtion_frame["phi"]))

            print(organ, cf, energy_per_desintegration, res_index)
            SFactorFrame.loc[res_index,("Sfactors")] = energy_per_desintegration
            SFactorFrame.loc[res_index,("CFs")] = CF_string
            SFactorFrame.loc[res_index,("SiteName")] = organ
            res_index = res_index + 1

# Make the class-objects from the generated pandas

site_list = np.unique(SFactorFrame["SiteName"].values)

# Set a dict for the individual frames

skeletalsites = {}
CF_ICRP_values = {}
tbv_fractions = {}

for site in site_list:
    
    tmp_frame = (SFactorFrame[SFactorFrame["SiteName"]==site])
    tmp_frame = tmp_frame.set_index("CFs")
    
    ICRP_CF = mass_data[mass_data["SiteName"] == site]["ICRP-CF"].values[0]/1.0e2
    
    tbv_correction = mass_data[mass_data["SiteName"] == site]["TBV-fraction"].values
    
    skeletalsites[site] = tmp_frame
    CF_ICRP_values[site] = ICRP_CF
    tbv_fractions[site] = tbv_correction
    
    print(ICRP_CF)
    
MaleSFactor = SkeletalSFactors(
    isotopename = "177-Lu",
    phantomsex = "M",
    skeletalfactors = skeletalsites,
    icrp_cf_vals=CF_ICRP_values,
    trabecular_corrections = tbv_fractions)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(SFactorFrame["Sfactors"], 'o')

plt.show()

In [ ]:

for organ in organ_list:

    ICRP_CF = mass_data[mass_data["SiteName"] == organ]["ICRP-CF"].values[0]/1.0e2
    print(ICRP_CF)

In [ ]:
print(SFactorFrame[SFactorFrame["SiteName"] == "LumbarVertebrae"])

In [ ]:
foo = (MaleSFactor.skeletalfactors["LumbarVertebrae"])
cf = (MaleSFactor.icrp_cf_vals["LumbarVertebrae"])
tbc_corr = (MaleSFactor.trabecular_corrections["LumbarVertebrae"])

indexes = foo.index

E = (foo[foo.index == "ICRP"]["Sfactors"]).values*1.60217662e-13
A_tilde = 812.2852129 # MBqHr

print(E)

cum_act = A_tilde*3600*1e6

energy_dep = E*cum_act

mass = 77.97*cf*(1-tbc_corr)*1e-3

print(1-tbc_corr)
print(mass)
print(cf)

#mass = 49.74*0.7*1e-3
print(energy_dep/mass)


In [ ]:
# Lets test with the lymphoma-data:

test_data_male = pd.read_excel("test_of_absorbed_dose.xlsx", sheet_name="lumbarDataBetalutin")

test_data_male["AbsDoseTest"] = np.zeros(len(test_data))


for i in range(len(test_data)):
    Cum = (test_data_male["MBqHr"][i])
    mass = (test_data_male["Mass"][i])
    
    Volume = mass*1e-3
    mass = cf*1.03*Volume
    
    print("Cumulative over mass: " + str(1e-5*Cum/mass))
    
    test_data_male["AbsDoseTest"][i] = (Cum*Energy_per_desintegration*1.602e-13*1e6*3600/mass)
    
print(test_data_male)
    
print(ss.pearsonr(test_data_male["AbsDoseTest"], test_data_male["PBN Neut"]))
print(ss.shapiro(test_data_male["PBN"]))


In [ ]:
# Lets test with the lymphoma-data:

test_data_female = pd.read_excel("test_of_absorbed_dose.xlsx", sheet_name="lumbarDataBetalutin")

test_data_female["AbsDoseTest"] = np.zeros(len(test_data))


for i in range(len(test_data)):
    Cum = (test_data_female["MBqHr"][i])
    mass = (test_data_female["Mass"][i])
    
    Volume = mass*1e-3
    Cum
    mass = cf*1.03*Volume
    
    test_data_female["AbsDoseTest"][i] = (Cum*Energy_per_desintegration*1.602e-13*1e6*3600/mass)
    
print(test_data_female)
    
print(ss.pearsonr(test_data_female["AbsDoseTest"], test_data_female["PBN Neut"]))

In [ ]:
#print(test_data_male)
#print(test_data_female)

foobar = pd.DataFrame()
foobar["PBN"] = test_data_female["PBN"]
foobar["PBN Neut"] = test_data_female["PBN Neut"]

print(test_data_male)
print(test_data_female)

foobar["Dose"] = np.zeros(len(foobar))


for i in range(len(foobar)):
    if (test_data_male["Sex"][i]) == "F":
        foobar["Dose"][i] = test_data_female["AbsDoseTest"][i]
        
    else:
        print("Itsa a boy!")
        foobar["Dose"][i] = test_data_male["AbsDoseTest"][i]

print(foobar)

print(test_data_female["AbsDoseTest"]-test_data_male["AbsDoseTest"])
print(test_data_female["AbsDoseTest"]-test_data_male["AbsDoseTest"])

print(ss.pearsonr(test_data_male["AbsDoseTest"], test_data_male["PBN Neut"]))
print(ss.pearsonr(test_data_male["AbsDoseTest"], test_data_male["PBN Neut"]))
print(ss.pearsonr(foobar["Dose"], foobar["PBN Neut"]))

In [ ]:
print((0.113-0.1012)/0.1012)

In [ ]:
energies = pd.read_excel("utils/decdataenergies.xlsx", sheet_name="AllElectrons")
saf_values = pd.read_excel("utils/saf_vals_male_table.xlsx")

energy_look_up_vector = saf_values["Energy"].values

print(energy_look_up_vector)

print(saf_values)



In [ ]:
# Set up a table for CF-100, using all the
# electron energies from DECDATA

len_of_table = len(energies)
print(len_of_table)

cf_factors = np.array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 0.7])

energy_absorbtion_frame = pd.DataFrame(columns=["Energy", "Y", "phi"], 
                                       index=np.arange(len_of_table))

energy_absorbtion_frame["Energy"] = energies["E (MeV)"]
energy_absorbtion_frame["Y"] = energies["Y (/nt)"]

for cf in cf_factors:

    CF_string = (str(int(cf*100))+"CF")

    for i in range(len_of_table):

        energy_to_look_up = (energy_absorbtion_frame.at[i, "Energy"])

        index = (np.abs(energy_look_up_vector-energy_to_look_up)).argmin()

        phi = (saf_values.at[index, CF_string])

        energy_absorbtion_frame.at[i, "phi"] = phi
        #energy_absorbtion_frame.at[i, "phi"] = 1.0

    s_factor = (np.sum(energy_absorbtion_frame["Energy"]*energy_absorbtion_frame["Y"]*energy_absorbtion_frame["phi"]))

    #print((s_factor))
    print(s_factor*1.602e-13)

In [ ]:
energy_per_desintegration = 4.5126e-15

s_factor = energy_per_desintegration

print(s_factor)

mass_am = (1.03*((1-0.1025)*0.1)/1e3)
#mass_am = 50*(1-0.135)*0.7*1.03*1e-3


Dose_1_MBqHrs = 3600*1e6*s_factor/mass_am

print(Dose_1_MBqHrs, " Factor")

old_factor = 0.0724/(1-0.135)

print(100*(Dose_1_MBqHrs-old_factor)/old_factor, " Diff factor")

#print((Dose_1_MBqHrs-0.0724)/0.0724)

#energy_per_MBqhrs = energy_per_desintegration*3600*1e6

#print(energy_per_MBqhrs)



In [3]:
# Snippet to test the class-constructing function:

from generate_func import create_skeletal_phantom

I did run!
I produced something!


In [8]:
(energy_frame, cf_dict, tbv_fracs) = create_skeletal_phantom(
    saf_file_path="utils/saf_values_male_redone.xlsx",
    isotope_data_path="utils/decdataenergies.xlsx")

MaleFactor = SkeletalSFactors(
    isotopename = "177-Lu",
    phantomsex = "M",
    skeletalfactors = energy_frame,
    icrp_cf_vals=cf_dict,
    trabecular_corrections=tbv_fracs)

In [10]:
(energy_frame, cf_dict, tbv_fracs) = create_skeletal_phantom(
    saf_file_path="utils/saf_values_female_redone.xlsx",
    isotope_data_path="utils/decdataenergies.xlsx")

FemaleFactor = SkeletalSFactors(
    isotopename = "177-Lu",
    phantomsex = "F",
    skeletalfactors = energy_frame,
    icrp_cf_vals=cf_dict,
    trabecular_corrections=tbv_fracs)

In [11]:
print(MaleFactor.skeletalfactors["LumbarVertebrae"])
print(FemaleFactor.skeletalfactors["LumbarVertebrae"])

       Sfactors         SiteName
CFs                             
CF10   0.028169  LumbarVertebrae
CF20   0.040565  LumbarVertebrae
CF30   0.052782  LumbarVertebrae
CF40   0.065009  LumbarVertebrae
CF50   0.077089  LumbarVertebrae
CF60   0.089274  LumbarVertebrae
CF70   0.101277  LumbarVertebrae
CF80   0.113087  LumbarVertebrae
CF90   0.124969  LumbarVertebrae
CF100  0.136865  LumbarVertebrae
ICRP   0.101277  LumbarVertebrae
       Sfactors         SiteName
CFs                             
CF10   0.025904  LumbarVertebrae
CF20   0.038492  LumbarVertebrae
CF30   0.050910  LumbarVertebrae
CF40   0.063419  LumbarVertebrae
CF50   0.075857  LumbarVertebrae
CF60   0.088224  LumbarVertebrae
CF70   0.100338  LumbarVertebrae
CF80   0.112414  LumbarVertebrae
CF90   0.124639  LumbarVertebrae
CF100  0.136652  LumbarVertebrae
ICRP   0.100338  LumbarVertebrae
